In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from dotenv import load_dotenv

load_dotenv()

key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=key,model_name='gpt-4',temperature=0.5)
llm

C:\Users\amish\AppData\Local\Temp\ipykernel_18972\2848416455.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=key,model_name='gpt-4',temperature=0.5)


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002694B6AF100>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002694B6BD2B0>, model_name='gpt-4', temperature=0.5, model_kwargs={}, openai_api_key='sk-proj-orf4DgNGLQeTwjmKDPU3T3BlbkFJDec4YU2xuxRcHUcalwoN', openai_proxy='')

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain,LLMChain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import PyPDF2

In [5]:
response_json = {
    '1':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'response':'correct answer'
    },
    '2':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'response':'correct answer'
    },
    '3':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'response':'correct answer'
    },
}

In [6]:
TEMPLATE='''
Text:{text}
You are an expert MCQ maker. Given bthe above text , it is your job to\
create a quiz of {number} multiple choice question for {subject} students to {tone} tone.
Make sure the question are not repeated and check all the question to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON and use it as a guide.\
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
'''

In [7]:
quiz_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [8]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key='quiz',verbose=True)

C:\Users\amish\AppData\Local\Temp\ipykernel_18972\1033466357.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key='quiz',verbose=True)


In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [11]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)

In [12]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','response_json'],
                                          output_variables=['quiz','review'],verbose=True)

In [56]:
file_path = 'your_file_path'

In [14]:
reader = PyPDF2.PdfReader(file_path)
page = reader.pages[2]

In [15]:
import PyPDF2

# Open the PDF file
with open(file_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Initialize a variable to hold the combined text
    combined_text = ""
    
    # Iterate through all the pages
    for page_num in range(len(reader.pages)):
        # Get a specific page
        page = reader.pages[page_num]
        
        # Extract text from the page
        text = page.extract_text()
        
        # Add the text to the combined text variable
        combined_text += text + "\n"  # Adding a newline for separation between pages

# Print the combined text
print(combined_text)


Advanced AI
Dr.Vaishnaw G.Kale
Associate Professor
Department of Computer Science 
Engineering & Applications
D.Y.Patil International University, Pune
Name of the subject:Advanced AI 
Course Code:AIM703
Programme:B.Tech
Year of study:Final Year
Semester :VII
Specialization:Artificial Intelligence Track 
Academic Year:2023 -24
About the Course
Syllabus
Module -I: Knowledge Representation and Reasoning
Introduction toAI:Foundations ofArtificial Intelligence, History ofArtificial Intelligence, State
oftheArt,Risks andBenefits ofAI,Intelligent Agents, Production systems inAI,
Review ofProblem Solving
Propositional Logic :Propositional Theorem Proving, Agents Based onPropositional Logic, First-
Order Logic, Syntax andSemantics ofFirst-Order Logic, Using First-Order Logic, Knowledge
Engineering inFirst-Order Logic, knowledge representation using propositional and predicate
logic, comparison ofpropositional andpredicate logic, reasoning,types ofReasoning
Advanced Artificial Intelligence
Next 

In [16]:
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "response": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "response": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "response": "correct answer"}}'

In [36]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            'text':combined_text,
            'number':5,
            'subject':'Artifical Intelligence',
            'tone':"Simple",
            'response_json':json.dumps(response_json)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Advanced AI
Dr.Vaishnaw G.Kale
Associate Professor
Department of Computer Science 
Engineering & Applications
D.Y.Patil International University, Pune
Name of the subject:Advanced AI 
Course Code:AIM703
Programme:B.Tech
Year of study:Final Year
Semester :VII
Specialization:Artificial Intelligence Track 
Academic Year:2023 -24
About the Course
Syllabus
Module -I: Knowledge Representation and Reasoning
Introduction toAI:Foundations ofArtificial Intelligence, History ofArtificial Intelligence, State
oftheArt,Risks andBenefits ofAI,Intelligent Agents, Production systems inAI,
Review ofProblem Solving
Propositional Logic :Propositional Theorem Proving, Agents Based onPropositional Logic, First-
Order Logic, Syntax andSemantics ofFirst-Order Logic, Using First-Order Logic, Knowledge
Engineering inFirst-Order Logic, knowledge representation using propositional and predicate
logic, compa

In [37]:
print('Prompt Token:',cb.prompt_tokens)
print('output Tokens:',cb.completion_tokens)
print('total tokens:',cb.total_tokens)
print('Total cost:',cb.total_cost)

Prompt Token: 7858
output Tokens: 723
total tokens: 8581
Total cost: 0.27912


In [38]:
response

{'text': 'Advanced AI\nDr.Vaishnaw G.Kale\nAssociate Professor\nDepartment of Computer Science \nEngineering & Applications\nD.Y.Patil International University, Pune\nName of the subject:Advanced AI \nCourse Code:AIM703\nProgramme:B.Tech\nYear of study:Final Year\nSemester :VII\nSpecialization:Artificial Intelligence Track \nAcademic Year:2023 -24\nAbout the Course\nSyllabus\nModule -I: Knowledge Representation and Reasoning\nIntroduction toAI:Foundations ofArtificial Intelligence, History ofArtificial Intelligence, State\noftheArt,Risks andBenefits ofAI,Intelligent Agents, Production systems inAI,\nReview ofProblem Solving\nPropositional Logic :Propositional Theorem Proving, Agents Based onPropositional Logic, First-\nOrder Logic, Syntax andSemantics ofFirst-Order Logic, Using First-Order Logic, Knowledge\nEngineering inFirst-Order Logic, knowledge representation using propositional and predicate\nlogic, comparison ofpropositional andpredicate logic, reasoning,types ofReasoning\nAdvan

In [41]:
quiz = response.get('quiz').replace('###RESPONSE_JSON\n', '')

In [43]:
quiz=json.loads(quiz)

In [45]:
print(quiz.items())

dict_items([('1', {'mcq': 'What is the course code for Advanced AI?', 'options': {'a': 'AIM703', 'b': 'AIM503', 'c': 'AIM803', 'd': 'AIM903'}, 'response': 'a'}), ('2', {'mcq': 'What is the type of reasoning where conclusions may be invalidated by adding more information to the knowledge base?', 'options': {'a': 'Deductive Reasoning', 'b': 'Inductive Reasoning', 'c': 'Non-monotonic Reasoning', 'd': 'Monotonic Reasoning'}, 'response': 'c'}), ('3', {'mcq': 'What is a proposition in Propositional Logic?', 'options': {'a': 'A declarative statement which is either true or false', 'b': 'An expression consisting of variables with a specified domain', 'c': 'A form of logical reasoning', 'd': 'None of the above'}, 'response': 'a'}), ('4', {'mcq': 'What is the purpose of using Propositional Logic?', 'options': {'a': 'To analyze a statement, individually or compositely', 'b': 'To arrive at a conclusion using a limited set of facts', 'c': 'To deduce new information from logically related known info

In [47]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["response"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [50]:
quiz_table_data

[{'MCQ': 'What is the course code for Advanced AI?',
  'Choices': 'a: AIM703 | b: AIM503 | c: AIM803 | d: AIM903',
  'Correct': 'a'},
 {'MCQ': 'What is the type of reasoning where conclusions may be invalidated by adding more information to the knowledge base?',
  'Choices': 'a: Deductive Reasoning | b: Inductive Reasoning | c: Non-monotonic Reasoning | d: Monotonic Reasoning',
  'Correct': 'c'},
 {'MCQ': 'What is a proposition in Propositional Logic?',
  'Choices': 'a: A declarative statement which is either true or false | b: An expression consisting of variables with a specified domain | c: A form of logical reasoning | d: None of the above',
  'Correct': 'a'},
 {'MCQ': 'What is the purpose of using Propositional Logic?',
  'Choices': 'a: To analyze a statement, individually or compositely | b: To arrive at a conclusion using a limited set of facts | c: To deduce new information from logically related known information | d: To make presumptions about events that occur every day',
  

In [52]:
df = pd.DataFrame(quiz_table_data)

In [55]:
df.to_csv("mcq_question.csv")